In [1]:
import gzip
import csv
import json
import re
import os
import sys
import numpy as np
import pandas as pd
from collections import defaultdict
import dask.dataframe as dd

dir = {your file directory here}

In [3]:
def load_books(file_name, head = 500):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            d['series'] = ','.join([str(i) for i in d['series']])
            d.pop('similar)books')
            d.pop('authors')
            count += 1
            data.append(d)
            
            # break if reaches the 100th line
            if (head is not None) and (count > head):
                break
    return data

In [5]:
def lw_jcBooks(file_name, chunk=10, nrows=-1):
    csv_file = os.path.join(dir, 'books.csv')
    lines_to_read = 0
    with gzip.open(file_name) as lines:
        for l in lines:
            lines_to_read += 1
        print(lines_to_read)
    total = 0
    count = 0
    data = []
    with gzip.open(file_name) as fin, open(csv_file, 'w', encoding='utf-8', newline='\n') as csv:
        header = True
        for l in fin:
            if nrows == -1:
                d = json.loads(l)
                count += 1
                total += 1
                data.append(d)

                if (chunk is not None) and (count > chunk - 1):
                    df = pd.DataFrame(data)
                    count = 0
                    data.clear()
                    df = df.drop(['popular_shelves', 'asin', 'kindle_asin', 'is_ebook', 'edition_information', 'authors', 'similar_books', 'url', 'link', 'title_without_series', 'series', 'text_reviews_count', 'average_rating', 'ratings_count', 'work_id'], axis=1)
                    df['description'] = df['description'].str.strip().str.replace('\'', '').str.replace('\"', '').str.replace('\n', '')
                    df.to_csv(csv, header=header, index=0)
                    header = False
                
                # Last Chunk
                if total == lines_to_read:
                    df = pd.DataFrame(data)
                    df = df.drop(['popular_shelves', 'asin', 'kindle_asin', 'is_ebook', 'edition_information', 'authors', 'similar_books', 'url', 'link', 'title_without_series', 'series', 'text_reviews_count', 'average_rating', 'ratings_count', 'work_id'], axis=1)
                    df['description'] = df['description'].str.strip().str.replace('\'', '').str.replace('\"', '').str.replace('\n', '')
                    df.to_csv(csv, header=header, index=0)
            else:
                if total < nrows:
                    d = json.loads(l)
                    count += 1
                    total += 1
                    data.append(d)

                    if (chunk is not None) and (count > chunk - 1):
                        df = pd.DataFrame(data)
                        count = 0
                        data.clear()
                        df = df.drop(['popular_shelves', 'asin', 'kindle_asin', 'is_ebook', 'edition_information', 'authors', 'similar_books', 'url', 'link', 'title_without_series', 'series', 'text_reviews_count', 'average_rating', 'ratings_count', 'work_id'], axis=1)
                        df['description'] = df['description'].str.strip().str.replace('\'', '').str.replace('\"', '').str.replace('\n', '')
                        df.to_csv(csv, header=header, index=0)
                        header = False
                else:
                    break
    return total

In [6]:
lw_jcBooks(os.path.join(dir, 'goodreads_books.json.gz'), chunk=1000)

2360655


2360655

In [6]:
dd_bi = dd.read_csv(os.path.join(dir, 'books.csv'), dtype={'isbn': 'object',
       'isbn13': 'object',
       'ratings_count': 'float64',
       'text_reviews_count': 'float64',
       'work_id': 'float64'})
dd_bi

,isbn,country_code,language_code,description,format,publisher,num_pages,publication_day,isbn13,publication_month,publication_year,image_url,book_id,title
npartitions=32,,,,,,,,,,,,,,
,object,object,object,object,object,object,float64,float64,object,float64,float64,object,int64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
len(dd_bi)

2360655

In [8]:
dd_bi.dtypes

isbn                  object
country_code          object
language_code         object
description           object
format                object
publisher             object
num_pages            float64
publication_day      float64
isbn13                object
publication_month    float64
publication_year     float64
image_url             object
book_id                int64
title                 object
dtype: object

In [9]:
dd_bi.isna().sum().compute()

isbn                  983373
country_code             490
language_code        1060153
description           412249
format                646754
publisher             654372
num_pages             764133
publication_day      1024429
isbn13                780263
publication_month     882945
publication_year      599625
image_url                490
book_id                    0
title                     11
dtype: int64

In [10]:
dd_bi.head()

,isbn,country_code,language_code,description,format,publisher,num_pages,publication_day,isbn13,publication_month,publication_year,image_url,book_id,title
0,0312853122,US,NaN,NaN,Paperback,St. Martin's Press,256.0,1.0,9780312853129,9.0,1984.0,https://images.gr-assets.com/books/1310220028m...,5333265,W.C. Fields: A Life on Film
1,0743509986,US,NaN,"Anita Diamant's international bestseller ""The ...",Audio CD,Simon & Schuster Audio,NaN,1.0,9780743509985,10.0,2001.0,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,Good Harbor
2,NaN,US,eng,Omnibus book club edition containing the Ladie...,Hardcover,"Nelson Doubleday, Inc.",600.0,NaN,NaN,NaN,1987.0,https://images.gr-assets.com/books/1304100136m...,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,US,eng,Addie Downs and Valerie Adler were eight when ...,Hardcover,Atria Books,368.0,14.0,9780743294294,7.0,2009.0,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,Best Friends Forever
4,0850308712,US,NaN,NaN,NaN,NaN,NaN,NaN,9780850308716,NaN,NaN,https://images.gr-assets.com/books/1413219371m...,287140,Runic Astrology: Starcraft and Timekeeping in ...


In [11]:
format = dd_bi['format']

In [12]:
format.unique().compute()

0                                Paperback
1                                 Audio CD
2                                Hardcover
3                                      NaN
4                                    ebook
                       ...                
2233    Paperback, Direct Sales Comic Book
2234                                Thesis
2235                            Hardcovver
2236       historical fiction--young adult
2237                              eJournal
Name: format, Length: 2238, dtype: object

In [19]:
desc = dd_bi['description']

In [28]:
desc.str.len().max().compute()

64980.0

In [29]:
series = dd_bi['series']

In [30]:
series.head()

0            []
1            []
2    ['189911']
3            []
4            []
Name: series, dtype: object

In [31]:
series.str.count(',').max().compute()

3

In [65]:
series_df = dd_bi[['series']]

In [66]:
series_df

,series
npartitions=35,
,object
,...
...,...
,...
,...


In [67]:
series_df.index.compute().is_unique

False

In [74]:
seriesIndexDups = series_df.index.compute().duplicated()

In [78]:
seriesIndexDups

array([False, False, False, ...,  True,  True,  True])

In [79]:
series_df.compute()

,series
0,[]
1,[]
2,['189911']
3,[]
4,[]
...,...
23807,[]
23808,['943808']
23809,['513826']
23810,[]


In [82]:
series_df = series_df.compute()

In [83]:
type(series_df)

pandas.core.frame.DataFrame

In [89]:
series_df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            23802, 23803, 23804, 23805, 23806, 23807, 23808, 23809, 23810,
            23811],
           dtype='int64', length=2360000)

In [90]:
series_df.reset_index(inplace=True)

In [95]:
series_df.index.is_unique

True

In [96]:
series_df['count'] = series_df['series'].str.count(',')

In [97]:
series_df

,index,series,count
0,0,[],0
1,1,[],0
2,2,['189911'],0
3,3,[],0
4,4,[],0
...,...,...,...
2359995,23807,[],0
2359996,23808,['943808'],0
2359997,23809,['513826'],0
2359998,23810,[],0


In [98]:
series_df['count'].max()

3

In [99]:
series_df[series_df['count'] > 0]

,index,series,count
41,41,"['250807', '415703', '181883']",2
84,84,"['246830', '362583', '362581', '623032']",3
112,112,"['266759', '1096220']",1
137,137,"['981445', '981448']",1
152,152,"['179176', '232917']",1
...,...,...,...
2359862,23674,"['220785', '504702']",1
2359925,23737,"['146642', '268841', '702396']",2
2359927,23739,"['1131248', '146641', '702395']",2
2359987,23799,"['154538', '958071']",1


In [100]:
titles = dd_bi[['title', 'title_without_series']]

In [103]:
titles['equal'] = titles['title'].compute().equals(titles['title_without_series'].compute())

In [106]:
titles['equal'].value_counts().compute()

True    2360000
Name: equal, dtype: int64

In [110]:
# 7327624 8948723


2360000

In [131]:
def lw_jcSimilar(file_name, chunk=10, nrows=-1):
    csv_file = os.path.join(dir, 'similar.csv')
    lines_to_read = 0
    with gzip.open(file_name) as lines:
        for l in lines:
            lines_to_read += 1
        print(lines_to_read)
    total = 0
    count = 0
    data = []
    with gzip.open(file_name) as fin, open(csv_file, 'w', encoding='utf-8', newline='\n') as csv:
        header = True
        for l in fin:
            if nrows == -1:
                d = json.loads(l)
                keysToExtract = ['book_id', 'similar_books']
                dd = {key:d[key] for key in keysToExtract}
                count += 1
                total += 1
                data.append(dd)

                if (chunk is not None) and (count > chunk - 1):
                    df = pd.DataFrame(data)
                    df = df.explode('similar_books')
                    count = 0
                    data.clear()
                    df.to_csv(csv, header=header, index=0)
                    header = False
                
                # Last Chunk
                if total == lines_to_read:
                    df = pd.DataFrame(data)
                    df = df.explode('similar_books')
                    df.to_csv(csv, header=header, index=0)
            else:
                if total < nrows:
                    d = json.loads(l)
                    count += 1
                    total += 1
                    data.append(d)

                    if (chunk is not None) and (count > chunk - 1):
                        df = pd.DataFrame(data)
                        df = df.explode('similar_books')
                        count = 0
                        data.clear()
                        df.to_csv(csv, header=header, index=0)
                        header = False
                else:
                    break
    return total

In [132]:
lw_jcSimilar(os.path.join(dir, 'goodreads_books.json.gz'), chunk=1000)

2360655


2360655

In [133]:
sDF = pd.read_csv(os.path.join(dir, 'similar.csv'))

In [134]:
sDF

,book_id,similar_books
0,5333265,NaN
1,1333909,8709549.0
2,1333909,17074050.0
3,1333909,28937.0
4,1333909,158816.0
...,...,...
18071750,11419866,1835589.0
18071751,11419866,1739458.0
18071752,11419866,1534367.0
18071753,11419866,8662586.0


In [135]:
sDF.loc[sDF['similar_books'] == 19997]

,book_id,similar_books
19,7327624,19997.0
111733,510103,19997.0
1048907,498560,19997.0
1375495,642070,19997.0
1408485,764029,19997.0
...,...,...
16882245,1767797,19997.0
16997530,642068,19997.0
17081858,974191,19997.0
17522499,1148956,19997.0


In [136]:
sDF.loc[sDF['similar_books'] == 7327624]

,book_id,similar_books
732440,19997,7327624.0
791885,18459291,7327624.0
2604083,637996,7327624.0
4273428,882101,7327624.0
4413438,1569323,7327624.0
4609906,1883810,7327624.0
6957998,615940,7327624.0
7724358,1988046,7327624.0
7914113,3443882,7327624.0
8843130,3743837,7327624.0


In [138]:
dd_bi.loc[dd_bi['book_id'] == 7327624].compute()

,isbn,text_reviews_count,country_code,language_code,average_rating,description,format,publisher,num_pages,publication_day,isbn13,publication_month,publication_year,image_url,book_id,ratings_count,work_id,title
2,NaN,7.0,US,eng,4.03,Omnibus book club edition containing the Ladie...,Hardcover,"Nelson Doubleday, Inc.",600.0,NaN,NaN,NaN,1987.0,https://images.gr-assets.com/books/1304100136m...,7327624,140.0,8948723.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ..."


In [5]:
def lw_jcWrites(file_name, chunk=10, nrows=-1):
    csv_file = os.path.join(dir, 'writes.csv')
    lines_to_read = 0
    if nrows == -1:
        with gzip.open(file_name) as lines:
            for l in lines:
                lines_to_read += 1
            print(lines_to_read)
    total = 0
    count = 0
    data = []
    with gzip.open(file_name) as fin, open(csv_file, 'w', encoding='utf-8', newline='\n') as csv:
        header = True
        for l in fin:
            if nrows == -1:
                d = json.loads(l)
                out = {'book_id': d['book_id']}
                temp = d['authors']
                dd = defaultdict(list)
                for di in temp:
                    for k,v in di.items():
                        dd[k].append(v)
                for k,v in dd.items():
                    out[k] = v
                count += 1
                total += 1
                data.append(out)

                if (chunk is not None) and (count > chunk - 1):
                    df = pd.DataFrame(data)
                    df = df.explode(['author_id', 'role'])
                    count = 0
                    data.clear()
                    df.to_csv(csv, header=header, index=0)
                    header = False
                
                # Last Chunk
                if total == lines_to_read:
                    df = pd.DataFrame(data)
                    df = df.explode(['author_id', 'role'])
                    df.to_csv(csv, header=header, index=0)
            else:
                if total < nrows:
                    d = json.loads(l)
                    out = {'book_id': d['book_id']}
                    temp = d['authors']
                    dd = defaultdict(list)
                    for di in temp:
                        for k,v in di.items():
                            dd[k].append(v)
                    for k,v in dd.items():
                        out[k] = v
                    count += 1
                    total += 1
                    data.append(out)

                    if (chunk is not None) and (count > chunk - 1):
                        df = pd.DataFrame(data)
                        df = df.explode(['author_id', 'role'])
                        count = 0
                        data.clear()
                        df.to_csv(csv, header=header, index=0)
                        header = False
                else:
                    break
    return total

In [6]:
lw_jcWrites(os.path.join(dir, 'goodreads_books.json.gz'), chunk=1000)

2360655


2360655

In [7]:
wDF = pd.read_csv(os.path.join(dir, 'writes.csv'))

In [8]:
len(wDF)

3324158

In [9]:
wDF

,book_id,author_id,role
0,5333265,604031.0,NaN
1,1333909,626222.0,NaN
2,7327624,10333.0,NaN
3,6066819,9212.0,NaN
4,287140,149918.0,NaN
...,...,...,...
3324153,26168430,2448.0,NaN
3324154,26168430,3460250.0,NaN
3324155,2342551,82312.0,Editor
3324156,22017381,7789809.0,NaN


In [13]:
wDF['role'].value_counts()

Translator                    124663
Illustrator                   108321
Contributor                    86653
Editor                         78678
Narrator                       40139
                               ...  
Illustrator (cover)                1
Translator: Fatameh Jafari         1
s japanskoga prevela               1
Review                             1
Version                            1
Name: role, Length: 7403, dtype: int64

In [17]:
wDF[wDF['author_id'].isna()]

,book_id,author_id,role
2400,23699819,NaN,NaN
15165,2597774,NaN,NaN
20221,18521522,NaN,NaN
25353,28253116,NaN,NaN
30671,17796597,NaN,NaN
...,...,...,...
3292670,26206589,NaN,NaN
3293465,1944688,NaN,NaN
3304936,26253723,NaN,NaN
3309356,23777395,NaN,NaN


In [22]:
dd_int = dd.read_csv(os.path.join(dir, 'goodreads_interactions.csv'))

In [23]:
dd_int

,user_id,book_id,is_read,rating,is_reviewed
npartitions=68,,,,,
,int64,int64,int64,int64,int64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [24]:
bmap = pd.read_csv(os.path.join(dir, 'book_id_map.csv'))

In [25]:
bmap

,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361
...,...,...
2360645,2360645,19517100
2360646,2360646,18597299
2360647,2360647,18584882
2360648,2360648,18518801


In [26]:
dd_int.merge(bmap, left_on='book_id', right_on='book_id_csv')

,user_id,book_id_x,is_read,rating,is_reviewed,book_id_csv,book_id_y
npartitions=68,,,,,,,
,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [28]:
dd_int = dd_int.merge(bmap, left_on='book_id', right_on='book_id_csv')

In [29]:
dd_int

,user_id,book_id_x,is_read,rating,is_reviewed,book_id_csv,book_id_y
npartitions=68,,,,,,,
,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [ ]:
dd_int = dd_int.drop(['book_id_x', 'book_id_csv'], axis=1)

In [42]:
dd_int = dd_int.rename(columns={'book_id_y':'book_id'})

In [44]:
len(dd_int)

228648342

In [52]:
dd_int.to_csv(os.path.join(dir, 'interactions.csv'),  single_file=True, )

['C:/Users/Matthew Sitzer/Downloads/Data for Project/interactions.csv']

In [46]:
orig_int = dd.read_csv(os.path.join(dir, 'goodreads_interactions.csv'))

In [47]:
len(orig_int)

228648342

In [51]:
orig_int.isna().any().compute()

user_id        False
book_id        False
is_read        False
rating         False
is_reviewed    False
dtype: bool

In [57]:
orig_int.head()

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [53]:
new_int = dd.read_csv(os.path.join(dir, 'interactions.csv'))

In [54]:
len(new_int)

228648342

In [56]:
new_int.head()

,Unnamed: 0,user_id,is_read,rating,is_reviewed,book_id
0,0,0,1,5,0,12
1,1,201,1,3,0,12
2,2,339,1,3,1,12
3,3,683,1,5,0,12
4,4,1856,0,0,0,12


In [61]:
test_values = new_int.loc[new_int['book_id'] == 7327624].compute()

In [63]:
len(test_values)

58

In [23]:
def load_data(filename, val=None, col=None):
    count = 0
    data = []
    with gzip.open(filename) as fin:
        if not val and not col:
            for l in fin:
                d = json.loads(l)
                count += 1
                data.append(d)
        else:
            for l in fin:
                d = json.loads(l)
                v = d[col]
                if val in v:
                    count += 1
                    data.append(d)
    print(count)
    return data

In [24]:
test_book = load_data(os.path.join(dir, 'goodreads_books.json.gz'), '396821', 'book_id')

5


In [30]:
pd.DataFrame([test_book[0]])[['description']]

,description
0,


In [16]:
authors = load_data(os.path.join(dir, 'goodreads_book_authors.json.gz'))

829529


In [20]:
authors[:2]

[{'average_rating': '3.98',
  'author_id': '604031',
  'text_reviews_count': '7',
  'name': 'Ronald J. Fields',
  'ratings_count': '49'},
 {'average_rating': '4.08',
  'author_id': '626222',
  'text_reviews_count': '28716',
  'name': 'Anita Diamant',
  'ratings_count': '546796'}]

In [21]:
authors_df = pd.DataFrame(authors)

In [22]:
authors_df.head()

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [24]:
authors_df = authors_df.drop(['average_rating', 'text_reviews_count', 'ratings_count'], axis=1)

In [25]:
authors_df.head()

,author_id,name
0,604031,Ronald J. Fields
1,626222,Anita Diamant
2,10333,Barbara Hambly
3,9212,Jennifer Weiner
4,149918,Nigel Pennick


In [27]:
authors_df.to_csv(os.path.join(dir, 'authors.csv'), header=True, index=0)

In [30]:
genres = load_data(os.path.join(dir, 'goodreads_book_genres_initial.json.gz'))

2360655


In [31]:
genres[:2]

[{'book_id': '5333265',
  'genres': {'history, historical fiction, biography': 1}},
 {'book_id': '1333909',
  'genres': {'fiction': 219, 'history, historical fiction, biography': 5}}]

In [32]:
genres_df = pd.DataFrame(genres)

In [34]:
genres_df.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [35]:
genres_norm_df = pd.json_normalize(genres)

In [36]:
genres_norm_df.head()

,book_id,"genres.history, historical fiction, biography",genres.fiction,"genres.fantasy, paranormal","genres.mystery, thriller, crime",genres.poetry,genres.romance,genres.non-fiction,genres.children,genres.young-adult,"genres.comics, graphic"
0,5333265,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1333909,5.0,219.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7327624,NaN,8.0,31.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,6066819,NaN,555.0,NaN,10.0,NaN,23.0,NaN,NaN,NaN,NaN
4,287140,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN


In [37]:
genres_norm_df

,book_id,"genres.history, historical fiction, biography",genres.fiction,"genres.fantasy, paranormal","genres.mystery, thriller, crime",genres.poetry,genres.romance,genres.non-fiction,genres.children,genres.young-adult,"genres.comics, graphic"
0,5333265,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1333909,5.0,219.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7327624,NaN,8.0,31.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,6066819,NaN,555.0,NaN,10.0,NaN,23.0,NaN,NaN,NaN,NaN
4,287140,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2360650,3084038,7.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
2360651,26168430,NaN,1.0,NaN,4.0,NaN,NaN,NaN,1.0,NaN,NaN
2360652,2342551,NaN,NaN,NaN,NaN,14.0,NaN,1.0,7.0,1.0,NaN
2360653,22017381,NaN,NaN,NaN,2.0,NaN,13.0,NaN,NaN,NaN,NaN


In [48]:
melted = pd.melt(genres_norm_df, id_vars='book_id', var_name='genre', value_name='num_tagged')

In [49]:
melted.head()

,book_id,genre,num_tagged
0,5333265,"genres.history, historical fiction, biography",1.0
1,1333909,"genres.history, historical fiction, biography",5.0
2,7327624,"genres.history, historical fiction, biography",NaN
3,6066819,"genres.history, historical fiction, biography",NaN
4,287140,"genres.history, historical fiction, biography",NaN


In [50]:
melted.dtypes

book_id        object
genre          object
num_tagged    float64
dtype: object

In [51]:
melted[melted['book_id'] == '7327624']

,book_id,genre,num_tagged
2,7327624,"genres.history, historical fiction, biography",NaN
2360657,7327624,genres.fiction,8.0
4721312,7327624,"genres.fantasy, paranormal",31.0
7081967,7327624,"genres.mystery, thriller, crime",1.0
9442622,7327624,genres.poetry,1.0
11803277,7327624,genres.romance,NaN
14163932,7327624,genres.non-fiction,NaN
16524587,7327624,genres.children,NaN
18885242,7327624,genres.young-adult,NaN
21245897,7327624,"genres.comics, graphic",NaN


In [52]:
melted = melted.dropna()

In [53]:
melted[melted['book_id'] == '7327624']

,book_id,genre,num_tagged
2360657,7327624,genres.fiction,8.0
4721312,7327624,"genres.fantasy, paranormal",31.0
7081967,7327624,"genres.mystery, thriller, crime",1.0
9442622,7327624,genres.poetry,1.0


In [54]:
melted['genre'] = melted['genre'].str.slice_replace(stop=7)

In [55]:
melted

,book_id,genre,num_tagged
0,5333265,"history, historical fiction, biography",1.0
1,1333909,"history, historical fiction, biography",5.0
5,287141,"history, historical fiction, biography",9.0
9,287149,"history, historical fiction, biography",1.0
10,6066814,"history, historical fiction, biography",38.0
...,...,...,...
23606499,25356997,"comics, graphic",2.0
23606500,20653416,"comics, graphic",46.0
23606507,28928706,"comics, graphic",2.0
23606516,2527820,"comics, graphic",12.0


In [56]:
melted[melted['book_id'] == '7327624']

,book_id,genre,num_tagged
2360657,7327624,fiction,8.0
4721312,7327624,"fantasy, paranormal",31.0
7081967,7327624,"mystery, thriller, crime",1.0
9442622,7327624,poetry,1.0


In [57]:
melted.to_csv(os.path.join(dir, 'genres.csv'), header=True, index=0)